In [3]:
#imports

from fastai.collab import *
from fastai.tabular.all import *

path = untar_data(URLs.ML_100k)

In [4]:
ratings  = pd.read_csv(
    path/'u.data',
    delimiter= '\t',
    header=None,
    names= ['user','movie','rating','timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


Let's take various factors of the movies and assume and they range between -1 and 1. The positive number indicating a stronger match and negative number indicating a weaker match.

- Is Sci-fi
- Action
- Old movies

The representation of "The Rise of the Skywalker" is as follows:

In [5]:
rise_skywalker = np.array([0.98,0.9,-0.9])

We can represent a user who likes modern Sci-fi as follows:

In [6]:
user1 = np.array([0.9,0.8,-0.6])

In [7]:
(user1 * rise_skywalker).sum()

2.1420000000000003

In [8]:
#Creating Data loaders

movies  = pd.read_csv(
    path/'u.item',
    delimiter= '|',
    encoding='latin-1',
    header=None,
    usecols=(0,1),
    names= ['movie','title'],
    engine='python')

movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [9]:
#merge with the ratings table

ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [10]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

,user,title,rating
0,239,Star Wars (1977),5
1,900,Crimson Tide (1995),2
2,235,Everyone Says I Love You (1996),4
3,97,It's a Wonderful Life (1946),2
4,637,"People vs. Larry Flynt, The (1996)",2
5,680,Schindler's List (1993),5
6,745,Star Trek IV: The Voyage Home (1986),2
7,751,Father of the Bride Part II (1995),2
8,828,L.A. Confidential (1997),4
9,864,Citizen Kane (1941),5


In [11]:
#Getting the number of movies and users
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])

#Selecting number of latent factors
n_factors = 5

#Initializing user and movie factors
user_factors = torch.randn(n_users, n_factors)
movie_factors = torch.randn(n_movies, n_factors)

In [12]:
one_hot_3 = one_hot(3, n_users).float()
user_factors.t()@one_hot_3

tensor([ 0.8747,  1.8554, -0.8791, -0.5910, -0.8516])

In [13]:
user_factors[3]

tensor([ 0.8747,  1.8554, -0.8791, -0.5910, -0.8516])

Class object in Python

In [14]:
class Example:
  def __init__(self, a):
    self.a = a

  def say(self,x):
    return f'Hello {self.a},{x}.'

In [15]:
ex = Example('Nikhil')
ex.say(' it\'s been a while')

"Hello Nikhil, it's been a while."

In [16]:
#Creating the Dot Product module


class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors):
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)

    def forward(self, x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        return (users * movies).sum(dim=1)

In [17]:
x,y = dls.one_batch()
x.shape

torch.Size([64, 2])

In [18]:
# Creating a learner to optimize the model

model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func = MSELossFlat())

learn.fit_one_cycle(5,5e-3)

epoch,train_loss,valid_loss,time
0,1.333099,1.369923,00:08
1,1.029551,1.108482,00:07
2,0.860753,1.003745,00:07
3,0.758122,0.910566,00:07
4,0.712716,0.891426,00:07


In [19]:
#Forcing the model to make predictions between 0 and 5

class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)
        self.y_range = y_range

    def forward(self, x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        return sigmoid_range((users*movies).sum(dim=1), *self.y_range)

model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func = MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.919647,0.998161,00:07
1,0.657676,0.951606,00:07
2,0.452191,0.949941,00:07
3,0.362838,0.950883,00:07
4,0.346094,0.950145,00:07


In [20]:
#Adding Biases to the model

class DotProductBias(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.user_bias = Embedding(n_users, 1)
        self.movie_factors = Embedding(n_movies, n_factors)
        self.movie_bias = Embedding(n_movies, 1)
        self.y_range = y_range

    def forward(self, x):
        users = self.user_factors(x[:,0])
        movies = self.movie_factors(x[:,1])
        res = (users * movies).sum(dim=1, keedim=True)
        res += self.user_bias(x[:0]) + self.movie_bias(x[:,1])
        return sigmoid_range(res, *self.y_range)

model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func = MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.924950,1.017919,00:07
1,0.660447,0.957429,00:07
2,0.438978,0.943284,00:07
3,0.353885,0.946448,00:07
4,0.341899,0.944573,00:07


The model with bias performs worse than the model without. If we look at the validation loss, we can see that it stops improving after the 2nd epoch. This is a clear indication that the model is overfitting.

We can add weight decay or L2 regularization. This will encourage the weights to be as small as possible. The idea is that larger co-efficients cause large canyons in the parabola. The weight decay will help us make the parabola wider.

In [21]:
# Loss with weight decay

learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.383696,0.938948,00:07
1,0.410680,0.931499,00:07
2,0.369461,0.915778,00:07
3,0.328636,0.900162,00:07
4,0.307859,0.895190,00:07


# What are Embeddings? How are they implemented in PyTorch?

In [21]:
#Creating out own embedding module

